In [ ]:
from keras.utils import plot_model
model = models[5]['LSTM'][1]
plot_model(model, show_shapes=True, show_layer_names=True)
model.summary()

In [ ]:
try:
  import pandas_ta
except ImportError:
  !pip install -q gdown pandas_ta streamlit streamlit-pdf-viewer > /dev/null
  !gdown 1XoLFPCkuExIehc4yduL2W35VgK3xBZiC
  import pandas_ta as ta
!curl ipv4.icanhazip.com

from bs4 import BeautifulSoup
from datetime import date, timedelta
# from google.colab import drive
from plotly import express as px, graph_objects as go, io as pio, subplots
# from pypfopt import EfficientFrontier, expected_returns, risk_models --PyPortfolioOpt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from time import sleep
import gdown
import joblib
import numpy as np
import pandas as pd
import pandas_ta as ta
import psycopg2
import requests

# drive.mount('/content/drive')
!unzip -q web_app.zip

In [ ]:
!streamlit run About.py &> /dev/null & npx localtunnel --port 8501

In [ ]:
conn=psycopg2.connect(
    host='btt2qpilkzi0pxzq8vzv-postgresql.services.clever-cloud.com',
    port='50013',
    database='btt2qpilkzi0pxzq8vzv',
    user='ufkrh0a7c7sfgxfain5q',
    password='bRhBfciIHwDWwDnPV7vGezZrYDQbsm'
)
# conn.rollback()

cur=conn.cursor()
def querier(query):
  conn.rollback()
  cur.execute(query)
  conn.rollback()
  return cur.fetchall()

###2

In [ ]:
def scraper(symbol):
  params = {
      'Symbol':symbol,
      'StartDate':'01/01/2024',
      'EndDate':'05/31/2024',
      'PageIndex':1,
      'PageSize':3000
  }
  response = requests.get('https://s.cafef.vn/Ajax/PageNew/DataHistory/PriceHistory.ashx', params)
  return response.json()

In [ ]:
data = []
symbols = ['VN30INDEX']#, 'BVH', 'GAS', 'MSN', 'SSI', 'VCB']

for symbol in symbols:
  output = scraper(symbol)['Data']['Data']
  for i in output:
    _date = i['Ngay']
    symbol = symbol
    _open = i['GiaMoCua']
    high = i['GiaCaoNhat']
    low = i['GiaThapNhat']
    volume = i['KhoiLuongKhopLenh']
    close = i['GiaDongCua']
    data.append((_date, symbol, _open, high, low, volume, close))
  sleep(3)

In [ ]:
df = pd.DataFrame(data, columns = ['date', 'symbol', 'open', 'high', 'low', 'volume', 'close'])
print(df.shape, '\n')
print(df.isna().sum(), '\n')
df.info()
df.to_csv('VN30.csv', index=False)

In [ ]:
df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y')

close = df.pivot_table(index = 'date', columns = 'symbol', values = 'close')
close = close[close.index < '2024-06-01']
close.tail()

In [ ]:
stock = pd.read_csv('VN30.csv')
stock['date'] = pd.to_datetime(stock['date'], format='%d/%m/%Y')
stock['symbol'].replace('VN30INDEX', 'VN30', inplace=True)
stock = stock[stock.date > '2014-12-31']

holiday = pd.read_csv('Holiday.csv')
holiday['date'] = holiday.month + '-' + holiday.year.astype(str)
holiday['date'] = holiday.date.str.replace(' ', '-')
holiday['date'] = pd.to_datetime(holiday['date'])
holiday = holiday[holiday.date.between('2015-01-05', '2024-01-31')]
holiday.set_index('date', inplace=True)
holiday.holiday.replace({
    'Day off for Hung Kings Festival': 'Hung Kings Commemoration Day',
    'Day off for International Labor Day': 'International Labor Day',
    'Day off for International New Year\'s Day': 'International New Year Day',
    'Day off for Liberation Day/Reunification Day': 'Reunification Day',
    'Hung Kings Festival': 'Hung Kings Commemoration Day',
    'Hung Kings Festival holiday': 'Hung Kings Commemoration Day',
    'Independence Day Holiday': 'Independence Day',
    'Independence Day observed': 'Independence Day',
    'International New Year\'s Day': 'International New Year Day',
    'International New Year\'s Eve': 'International New Year Day',
    'Liberation Day/Reunification Day': 'Reunification Day',
    'Liberation Day/Reunification Day Holiday': 'Reunification Day',
    'New Year\'s Day Holiday': 'International New Year Day',
    'Tet holiday': 'Tet Holiday',
    'Vietnamese New Year': 'Tet Holiday',
    'Vietnamese New Year\'s Eve': 'Tet Holiday'
}, inplace=True)
holiday.drop(['month', 'year'], axis=1, inplace=True)

pivot = stock.pivot_table(index='date', columns='symbol', values=['open', 'high', 'low', 'volume', 'close'])
scaler = MinMaxScaler()

###3

In [ ]:
def skew_kurtosis(start_date, end_date):
  data = pivot.close.drop('VN30', axis=1)
  data = 100*data[(data.index >= start_date) & (data.index <= end_date)].pct_change()[1:]
  return pd.DataFrame({'mean':data.mean(), 'skewness':data.skew(), 'kurtosis':data.kurtosis()}, index=['BVH', 'GAS', 'MSN', 'SSI', 'VCB']).round(2)

skew_kurtosis('2023-01-01', '2023-12-31')

In [ ]:
pd.DataFrame(pivot.close.to_records()).drop(['date', 'VN30'], axis=1).pct_change().cov().round(5)

In [ ]:
pd.DataFrame(pivot.close.to_records()).drop('date', axis=1).corr().round(2)

In [ ]:
drop_indices = []
previous_day = holiday.index[0]
for d in holiday.index[1:]:
  if d == previous_day+timedelta(days=1):
    drop_indices.append(d)
  else:
    drop_indices.append(previous_day)
  previous_day = d

In [ ]:
def line_chart(data, start_date, end_date, symbol):
  data = data[(data.index >= start_date) & (data.index <= end_date)]
  fig = px.scatter(data, x=data.index, y=symbol, trendline='ols', trendline_color_override=False if type(symbol) == list else 'yellow', template='plotly_dark')
  if type(symbol) == str:
    for d in holiday[(~holiday.index.isin(drop_indices)) & (holiday.index.year==data.index.year[0])].index:
      fig.add_vline(x=d, label=dict(text=holiday.loc[d,'holiday']))
      # fig.add_vline(x=d, line=dict(width=250), opacity=0.1)
      fig.add_vrect(x0=d-timedelta(days=20), x1=d+timedelta(days=20), fillcolor='white', opacity=0.1)
  fig.update_traces(mode='lines', connectgaps=True)
  fig.update_layout(title=f'{symbol} Close From {start_date} To {end_date}', margin=dict(l=5, r=5, t=50, b=5), height=360, xaxis_title=None, yaxis_title=None, legend_title='Symbols')
  return fig

In [ ]:
line_chart(pivot.close, '2023-12-01', '2024-04-30', 'VN30').show()

#Đại dịch Covid-19 bắt đầu ảnh hưởng tới TTCK Việt Nam từ cuối tháng 1-2020, sớm hơn thế giới tới gần một tháng.
#Phản ứng lo sợ của nhà đầu tư trước các bất định do dịch bệnh gây ra đã dẫn đến một đợt sụt giảm nhanh và mạnh chưa từng thấy.
#Chỉ số VN30 chỉ trong hai tháng đã giảm gần 30% và xuống mức thấp nhất trong vòng bốn năm.
#Tuy nhiên, thị trường đã phục hồi nhanh chóng trong chín tháng còn lại với mức tăng trưởng hơn 75% kể từ đáy.
#Sự phục hồi của VN30 đến từ việc kiểm soát dịch bệnh tốt trong nước của Chính phủ, các biện pháp kích thích tiền tệ và tài khóa kịp thời, đặc biệt là dòng tiền “rẻ” nhờ môi trường lãi suất thấp đã kích thích một lượng vốn lớn đổ vào TTCK.

In [ ]:
line_chart(pivot.close, '2021-01-01', '2021-12-31', 'VN30').show()

#Năm 2021, chỉ số VN30 nhiều lần thiết lập đỉnh lịch sử mới, cao nhất là vào ngày 25/11 khi chạm mốc 1.572,46 điểm. Đây cũng là mức cao nhất trong lịch sử TTCK tính đến thời điểm này.
#Trước đó, ngay trong ngày đầu tiên của quý 2/2021, VN30 đã chính thức vượt mốc 1.200 điểm - mốc kỷ lục được thiết lập từ 2018.
#Tính đến ngày 31/12/2021, VN30 đạt 1.535,71 điểm, tăng hơn 43% so với cuối năm 2020.
#Làn sóng nhà đầu tư cá nhân mới tham gia thị trường là một trong những nhân tố quan trọng thúc đẩy tăng trưởng mạnh mẽ của thanh khoản trong giai đoạn này.
#Theo thống kê từ các công ty chứng khoán, năm 2021, người dân đã chuyển thêm khoảng 3 tỷ USD vào đầu tư chứng khoán.

In [ ]:
line_chart(pivot.close, '2022-01-01', '2022-12-31', 'VN30').show()

#Sau một năm 2021 thăng hoa, chỉ số VN30 tiếp tục duy trì ở mức cao trong 3 tháng đầu năm 2022 trước khi bắt đầu tụt dốc từ đầu tháng 4.
#Đến 30/12, VN30 xuống còn 1.005,19 điểm, thấp hơn 35% so với mức đỉnh hồi đầu tháng 4.

#Đà giảm của VN30 được lý giải bởi nhiều nguyên nhân như thị trường liên tục gặp những cú sốc như các vụ khởi tố lãnh đạo cấp cao tại Tập đoàn FLC, Tân Hoàng Minh, Vạn Thịnh Phát,...
#Đồng thời, một số yếu tố khác cũng ảnh hưởng đến thị trường như lãi suất tăng; thắt chặt tín dụng đối với các phân khúc cho vay rủi ro cao, bao gồm đầu tư bất động sản, chứng khoán,...
#Ngoài ra, tình hình quốc tế như cuộc chiến Nga - Ukraine khiến giá dầu thô tăng vọt, TQ áp dụng chính sách "Zero-COVID" dẫn đến chuỗi cung ứng bị đứt gãy, FED liên tục tăng lãi suất nhằm kiềm chế lạm phát,...
#Qua đó ảnh hưởng tâm lý các nhà đầu tư cá nhân - bộ phận chiếm đến 90% cơ cấu nhà đầu tư trên thị trường chứng khoán.

In [ ]:
line_chart(pivot.close, '2023-01-01', '2023-12-31', 'VN30').show()

#Thị trường nhìn chung diễn biến tích cực cho đến đầu tháng 9, có thời điểm VN30 đạt hơn 1.255 điểm.
#Lý do đến từ việc nới lỏng chính sách tiền tệ của Ngân hàng Nhà nước, với 4 lần cắt giảm lãi suất điều hành nhằm hỗ trợ tăng trưởng kinh tế.
#Nhà đầu tư cá nhân là động lực chính cho giai đoạn tăng điểm kéo dài trong 4 tháng, từ cuối tháng 4 đến hết tháng 8/2023, dù khối ngoại có động thái bán ròng.
#Dòng tiền cá nhân nhập cuộc thận trọng giai đoạn đầu, nhưng mạnh mẽ hơn từ cuối tháng 4 đã giúp VN30 dần đi lên, khiến VN trở thành một trong những chỉ số chứng khoán tăng điểm ấn tượng trên thế giới.

#Tuy nhiên, trong vòng 2 tháng, thành quả của giai đoạn trước gần như tiêu tan sau nhịp điều chỉnh chủ yếu do yếu tố ngoại biên.
#Áp lực tỷ giá, dẫn đến Ngân hàng Nhà nước thực hiện biện pháp can thiệp là rút một lượng lớn tiền về hệ thống thông qua kênh tín phiếu.
#Trong khi đó, khối ngoại duy trì động thái bán ròng. Tâm lý nhà đầu tư nội bị ảnh hưởng khiến chỉ số giảm điểm, dù yếu tố vĩ mô trong nước vẫn tích cực.
#(GDP quý III tăng 5,3%, quý IV tăng 6,72%, lạm phát được kiểm soát, lãi suất tiếp tục giảm,...)
#Giai đoạn cuối năm, VN30 dao động quanh ngưỡng 1.100 điểm, đóng cửa phiên 29/12 tại 1.131,46 điểm.
#Thị trường chứng khoán năm 2023 có độ nhạy cao với chính sách tiền tệ.
#Thị trường hưng phấn, tăng mạnh sau động thái cắt giảm lãi suất liên tiếp của Ngân hàng Nhà nước và phản ứng tiêu cực khi có động thái hút tiền từ cơ quan này để ổn định tỷ giá.
#Hai tháng cuối năm, sau khi Ngân hàng Nhà nước ngừng hút tiền, thanh khoản hệ thống ngân hàng ổn định trở lại, diễn biến thị trường chứng khoán hồi phục nhẹ rồi đi ngang.

In [ ]:
def candlestick_chart(data, start_date, end_date, symbol):
  data = data[(data.index >= start_date) & (data.index <= end_date)]
  fig = go.Figure([go.Candlestick(x=data.index, open=data.open[symbol], high=data.high[symbol], low=data.low[symbol], close=data.close[symbol], showlegend=False)])
  # fig.add_trace(go.Scatter(x=data.index, y=ta.sma(data.close[symbol], length), line_color='blue', name='SMA%s' %length))
  fig.add_trace(go.Scatter(x=data.index, y=ta.ema(data.close[symbol], 10), line_color='red', name='EMA10'))
  fig.add_trace(go.Scatter(x=data.index, y=ta.ema(data.close[symbol], 20), line_color='green', name='EMA20'))
  fig.add_trace(go.Scatter(x=data.index, y=ta.ema(data.close[symbol], 50), line_color='blue', name='EMA50'))
  fig.add_trace(go.Scatter(x=data.index, y=ta.bbands(data.close[symbol], 20).iloc[:,0], line_color='white', line={'dash':'dash'}, showlegend=False))
  fig.add_trace(go.Scatter(x=data.index, y=ta.bbands(data.close[symbol], 20).iloc[:,2], line_color='white', line={'dash':'dash'}, showlegend=False))
  fig.update_layout(title=f'{symbol} Volatility From {start_date} To {end_date}', margin=dict(l=5, r=5, t=50, b=5), height=360, template='plotly_dark', xaxis_rangeslider_visible=False)
  return fig

In [ ]:
candlestick_chart(pivot, '2023-07-01', '2023-12-31', 'VCB').show()

In [ ]:
def bar_chart(symbol):
  data = {'year':[], 'change':[]}
  for year in range(2015, 2024):
    data['year'].append(year)
    year_open = float(pivot[pivot.index == pivot[pivot.index.year == year].index.min()].open[symbol])
    year_close = float(pivot[pivot.index == pivot[pivot.index.year == year].index.max()].close[symbol])
    data['change'].append(100*(year_close/year_open - 1))
  data = pd.DataFrame(data)
  fig = px.line(data, x='year', y='change')
  fig['data'][0]['line']['color'] = 'white'
  fig.add_bar(x=data.year, y=data.change, text=data.change.round(2), textposition='outside')
  fig.update_traces(marker_color=np.where(np.array(data['change']) > 0, 'green', 'red'), showlegend=False, textfont_size=8)
  return fig

In [ ]:
fig = subplots.make_subplots(rows=2, cols=5, vertical_spacing=0.1, specs=[[{}, {}, {}, {}, {}], [None, {'colspan':3}, None, None, None]], subplot_titles=pivot.close.columns)
for s, i in zip(['BVH', 'GAS', 'MSN', 'SSI', 'VCB'], [1, 2, 3, 4, 5]):
  fig.add_trace(bar_chart(s).data[0], row=1, col=i)
  fig.add_trace(bar_chart(s).data[1], row=1, col=i)
fig.add_trace(bar_chart('VN30').data[0], row=2, col=2)
fig.add_trace(bar_chart('VN30').data[1], row=2, col=2)
fig.update_layout(title='Stock Price Change Each Year', margin=dict(l=5, r=5, t=75, b=5), template='plotly_dark')
fig.show()

In [ ]:
data = pd.DataFrame(scaler.fit_transform(pivot.close), columns=['BVH', 'GAS', 'MSN', 'SSI', 'VCB', 'VN30'], index=pivot.index)
line_chart(data, '2015-01-01', '2023-12-31', ['BVH', 'GAS', 'MSN', 'SSI', 'VCB']).show()

In [ ]:
drop_indexes0 = []
previous_day = holiday.index[-1]
for d in holiday.index[-2::-1]:
  if d == previous_day-timedelta(days=1):
    drop_indexes0.append(d)
  previous_day = d

In [ ]:
line_chart(pivot.close-pivot.open, '2020-01-01', '2020-12-31', 'BVH').show()
line_chart(pivot.volume, '2020-01-01', '2020-12-31', 'BVH').show()

###4

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import AveragePooling1D, Conv1D, Dense, Dropout, Flatten, GlobalAveragePooling1D, Input, LayerNormalization, LSTM, MaxPooling1D, MultiHeadAttention, SimpleRNN
from keras.models import load_model, Model, Sequential
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score
import keras
import keras.backend as K
models = {5:{}, 10:{}, 20:{}}

In [ ]:
# from sklearn.model_selection import train_test_split
model_list = ['CNN', 'RNN', 'LSTM']
timestep = 5
epochs = 8
batch_size = 8
scaler = MinMaxScaler()
def input_transform(input, timestep, transform_only=2):
  X = []
  y = []
  if transform_only == 1: data = scaler.transform(input).squeeze()
  else: data = scaler.fit_transform(input).squeeze()
  for start_index in range(len(data)):
    end_index = start_index + timestep
    if end_index > len(data) - 1:
      break
    X.append(data[start_index:end_index])
    y.append(data[end_index])
  X, y = np.array(X), np.array(y)
  return X.reshape(X.shape[0], X.shape[1], 1), y.reshape(-1,1)

input = pd.concat([pivot.close[['VN30']][:-22], close[:]])[['VN30']].dropna()
X, y = input_transform(input, 5)
X_train, y_train = X[:1724], y[:1724] #input_transform(input[:-59], timestep)
X_test, y_test = X[1724:], y[1724:] #input_transform(input[-59:], timestep, 1)

def rmse(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_true-y_pred)))

joblib.dump(scaler, 'MinMaxScaler.sav')

In [ ]:
input.shape

In [ ]:
# !mv *.py ./pages
# conn.rollback()
cur.execute(open('PostgreSQL_query.txt', 'r').read())
conn.rollback()
result = pd.DataFrame(cur.fetchall(), columns=['date', 'trader_id', 'symbol', 'price', 'buy_quantity', 'sell_quantity'])
result = result.astype({'trader_id':str, 'price':int})
result['date'] = pd.to_datetime(result.date, format='%Y-%m-%d').dt.date
result

# fig = px.line(result, x='date', y=['buy_quantity', 'sell_quantity'])
# fig.update_layout(legend_title=None, legend=dict(x=0, y=1, bordercolor='White', borderwidth=2), margin=dict(l=5, r=5, t=5, b=5), xaxis=dict(showgrid=False), xaxis_title=None, yaxis_title=None, template='plotly_dark', height=360)
# fig.show()

# blackbox.ai/chat/YIxl8Z3

In [ ]:
# @title
prompt = f'''From now on, you will play a role as a senior analyst in stock trading with Python and its module pandas_ta are your main analysis tools. You are capable of decoding complicated technical indicators and unpredictable market movements. Your jobs now are to provide detailed explanations and clear insights from given figures; make possible predictions and offer valuable observations.
What the price trend of this stock will be in the next few days?
Data today of BVH:
- Close: {figures.iloc[-3,0]}
- EMA_10: {figures.iloc[-3,1]}
- EMA_20: {figures.iloc[-3,2]}
- BBL_20: {figures.iloc[-3,3]}
- BBM_20: {figures.iloc[-3,4]}
- BBU_20: {figures.iloc[-3,5]}
- RSI_20: {figures.iloc[-3,8]}
- CCI_20: {figures.iloc[-3,9]}
Data yesterday of BVH:
- Close: {figures.iloc[-4,0]}
- EMA_10: {figures.iloc[-4,1]}
- EMA_20: {figures.iloc[-4,2]}
- BBL_20: {figures.iloc[-4,3]}
- BBM_20: {figures.iloc[-4,4]}
- BBU_20: {figures.iloc[-4,5]}
- RSI_20: {figures.iloc[-4,8]}
- CCI_20: {figures.iloc[-4,9]}'''
print(prompt)

In [ ]:
x = np.arange(len(input))
fig = go.Figure(go.Scatter(x=x[:len(y_train)], y=y_train.squeeze(), name='Train data'))
fig.add_trace(go.Scatter(x=x[len(y_train):], y=y_test.squeeze(), name='Test data'))
fig.update_layout(
    legend_title=None,
    margin=dict(l=5, r=5, t=5, b=5),
    xaxis=dict(showgrid=False),
    xaxis_title=None,
    yaxis_title=None,
    template='plotly_dark',
    height=360, width=720
)
fig.show()

In [ ]:
def model_selection(model_name, timestep):
#model1 = Sequential()
#model1.add(Conv1D(filters=50, kernel_size=2, activation='relu', input_shape=(timestep,1)))
#model1.add(MaxPooling1D(pool_size=2))
#model1.add(Flatten())
#model1.add(Dense(100, activation='relu'))
#model1.add(Dense(1))

#model2 = Sequential()
#model2.add(SimpleRNN(50, activation='tanh', return_sequences=True, input_shape=(timestep,1)))
#model2.add(Dropout(0.2))
#model2.add(SimpleRNN(50, activation='tanh', return_sequences=True))
#model2.add(SimpleRNN(50, activation='tanh', return_sequences=True))
#model2.add(SimpleRNN(50))
#model2.add(Dense(1))

#model3 = Sequential()
#model3.add(LSTM(128, return_sequences=True, input_shape=(timestep,1)))
  # model3.add(Dropout(0.2))
#model3.add(LSTM(64, return_sequences=False))
  # model3.add(Dropout(0.2))
#model3.add(LSTM(25))
  # model3.add(Dropout(0.2))
#model3.add(Dense(1))

#model4 = Sequential()

  if model_name == 'CNN': return model1, model_name
  elif model_name == 'RNN': return model2, model_name
  elif model_name == 'LSTM': return model3, model_name
  elif model_name == 'Transformer': return model4, model_name

def model_running(model_selection, model_name):
  model_selection.compile(loss='mse', optimizer='adam')
  es = EarlyStopping(patience=5, verbose=1)
  mc = ModelCheckpoint(f'{timestep}_{model_name}_model_{epochs}-{batch_size}.keras', save_best_only=True, verbose=0)
  history = model_selection.fit(X_train, y_train, callbacks=[mc], epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

  return history

for timestep in [5, 10, 20]:
  X, y = input_transform(input, timestep)
  X_train, y_train = X[:1724], y[:1724]
  for m in model_list:
    print(m)
    model, model_name = model_selection(m, timestep)
    history = model_running(model, model_name)
    models[timestep].update({m:[history, model]})

joblib.dump(models, f'Models_using_MinMaxScaler_{epochs}-{batch_size}.sav')
!rm *.keras

In [ ]:
buy_quantity = []
sell_quantity = []
while 1:
  b_q_v = np.random.choice(range(0,700,100), 77)
  s_q_v = np.random.choice(range(0,1000,100), 77)
  if (-1 < b_q_v.sum() - s_q_v.sum() < 501) and (s_q_v[0] <= b_q_v[0]):
    buy_quantity.append(b_q_v)
    sell_quantity.append(s_q_v)
    print(len(buy_quantity), end=' ')
  if len(sell_quantity) == 5: break
# np.concatenate(buy_quantity).sum() - np.concatenate(sell_quantity).sum()

In [ ]:
cur.execute('SELECT symbol, SUM(buy_quantity-sell_quantity), SUM(price*(sell_quantity-buy_quantity)) FROM "log" GROUP BY ROLLUP(symbol);')
conn.rollback()
pd.DataFrame(cur.fetchall(), columns=['symbol', 'number of shares left', 'earn/lose'])

In [ ]:
for t in [('BVH',0), ('GAS',1), ('MSN',2), ('SSI',3), ('VCB',4)]:
  for (d, b_q, s_q) in zip(result[result.symbol==t[0]].date.sort_values(), buy_quantity[t[1]], sell_quantity[t[1]]):
    cur.execute(f'UPDATE "log" SET buy_quantity={b_q}, sell_quantity={s_q} WHERE symbol=\'{t[0]}\' AND trading_date=\'{d}\'')
    conn.commit()

In [ ]:
custom_objects={'rmse':rmse}
with keras.utils.custom_object_scope(custom_objects):
  models = joblib.load('Models_using_MinMaxScaler_8-8.sav')
fig = subplots.make_subplots(rows=3, cols=3, vertical_spacing=0.05, horizontal_spacing=0.03, subplot_titles=model_list, shared_xaxes=True)
for model_sel, col in zip(model_list, [1,2,3]):
  history = models[5][model_sel][0]
  fig.add_trace(px.line(y=[history.history['loss'], history.history['val_loss']]).data[0], row=1, col=col)
  fig.add_trace(px.line(y=[history.history['loss'], history.history['val_loss']]).data[1], row=1, col=col)

  history = models[10][model_sel][0]
  fig.add_trace(px.line(y=[history.history['loss'], history.history['val_loss']]).data[0], row=2, col=col)
  fig.add_trace(px.line(y=[history.history['loss'], history.history['val_loss']]).data[1], row=2, col=col)

  history = models[20][model_sel][0]
  fig.add_trace(px.line(y=[history.history['loss'], history.history['val_loss']]).data[0], row=3, col=col)
  fig.add_trace(px.line(y=[history.history['loss'], history.history['val_loss']]).data[1], row=3, col=col)
fig.update_layout(showlegend=False, margin=dict(l=5, r=5, t=30, b=5), template='plotly_dark', height=480)
fig.for_each_yaxis(lambda y: y.update(showgrid=False))
fig.show()

In [ ]:
def model_evaluation(y_test_true, y_test_pred, model_name):
  print(model_name)
  print('MAE: %.2f' %mae(y_test_true, y_test_pred))
  print('RMSE: %.2f' %np.sqrt(mse(y_test_true, y_test_pred)))
  print('R^2: %.2f' %r2_score(y_test_true, y_test_pred))

In [ ]:
timestep = 5
X, y = input_transform(input, timestep, 1)
X_test, y_test = X[1724:], y[1724:]
fig = subplots.make_subplots(rows=1, cols=3, horizontal_spacing=0.03, subplot_titles=model_list, shared_yaxes=True)
for model_sel, col in zip(model_list, [1,2,3]):
  model = models[timestep][model_sel][1]
  # y_train_pred = model.predict(X_train)
  # y_train_pred = scaler.inverse_transform(y_train_pred)
  # y_train_true = scaler.inverse_transform(y_train)

  y_test_pred = model.predict(X_test)
  # y_test_pred += np.std(X_test, axis=1)/5
  y_test_pred = scaler.inverse_transform(y_test_pred)
  y_test_true = scaler.inverse_transform(y_test)

  fig.add_trace(px.line(y=[y_test_pred.squeeze(), y_test_true.squeeze()]).data[0], row=1, col=col)
  fig.add_trace(px.line(y=[y_test_pred.squeeze(), y_test_true.squeeze()]).data[1], row=1, col=col)
  model_evaluation(y_test_true, y_test_pred, model_sel)
fig.update_layout(title=timestep, showlegend=False, margin=dict(l=5, r=5, t=50, b=5), template='plotly_dark', height=240)
for figure in fig.data:
  figure['x'] = input.index[-len(X_test):]
# fig.show()

In [ ]:
def ai_ta(prompt):
  payload = {
      'agentMode': {},
      'codeModelMode': True,
      'githubToken': None,
      'id': 'Jslb1W7',
      'isChromeExt': False,
      'isMicMode': False,
      'messages': [{'id': 'Jslb1W7', 'content': prompt, 'role': 'user'}],
      'previewToken': None,
      'trendingAgentMode': {},
      'userId': '87d93ba7-d13d-4caa-a348-bf0f0200301b'
  }
  response = requests.post('https://www.blackbox.ai/api/chat', json=payload)
  return response.text
response = ai_ta(prompt)
print(response)

###5

In [ ]:
with pd.ExcelWriter('Stock_price.xlsx') as writer:
  for col in ['open', 'high', 'low', 'close', 'volume']:
    pd.DataFrame(pivot[col].to_records()).to_excel(writer, sheet_name=col, index=False)

In [ ]:
def model_selection(model_name, timestep):
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(timestep,1)))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(AveragePooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(timestep,1)))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(AveragePooling1D())
model.add(Flatten())
model.add(Dense(1))

model = Sequential()
model.add(SimpleRNN(32, activation='relu', return_sequences=True, input_shape=(timestep,1)))
model.add(SimpleRNN(32, activation='relu', return_sequences=True))
model.add(SimpleRNN(64, activation='relu', return_sequences=True))
model.add(SimpleRNN(64))
model.add(Dense(1))

model = Sequential()
model.add(LSTM(32, activation='relu', return_sequences=True, input_shape=(timestep,1)))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1))